Update commissioners after election results.

In [1]:
import os
os.chdir('../')

import pytz
import pandas as pd
from datetime import datetime

from scripts.common import list_commissioners

In [2]:
people = pd.read_csv('data/people.csv')
commissioners = list_commissioners()
results = pd.read_csv('data/results.csv')
candidates = pd.read_csv('data/candidates.csv')
write_in_winners = pd.read_csv('data/write_in_winners.csv')

In [3]:
results_cand = pd.merge(results, candidates, how='inner', on=['candidate_id', 'smd_id'])
results_cand_people = pd.merge(results_cand, people, how='inner', on='person_id')
results_cand_people_comm = pd.merge(
    results_cand_people, commissioners, how='inner', on=['person_id', 'smd_id'])

Current commissioners who won re-election. Their `end_date` in the Google Sheet should be set to `2023-01-02`

In [4]:
won_reelection = results_cand_people_comm.loc[
    (results_cand_people_comm.winner) & (results_cand_people_comm.is_current)
    , ['person_id','smd_id','full_name', 'end_date']] #.to_clipboard(index=False)

won_reelection

,person_id,smd_id,full_name,end_date
1,10003,smd_1A03,Zach Rybarczyk,2023-01-02 00:00:00-05:00
2,10005,smd_1A05,Christine Miller,2023-01-02 00:00:00-05:00
5,10008,smd_1A08,Kent C. Boese,2023-01-02 00:00:00-05:00
6,10009,smd_1A09,Michael Wray,2023-01-02 00:00:00-05:00
7,10010,smd_1A10,Rashida Brown,2023-01-02 00:00:00-05:00
...,...,...,...,...
283,10149,smd_5A05,Ronnie Edwards,2023-01-02 00:00:00-05:00
285,10232,smd_7D02,Siraaj M. Hasan,2023-01-02 00:00:00-05:00
287,10016,smd_1B04,Deborah Thomas,2023-01-02 00:00:00-05:00
289,10119,smd_4A07,Marlene Moss,2023-01-02 00:00:00-05:00


Current commissioners who lost re-election. Their `end_date` in the Google Sheet should remain `2021-01-02`. 

In [5]:
lost_reelection = results_cand_people_comm.loc[
    ~(results_cand_people_comm.winner) & (results_cand_people_comm.is_current)
    , ['person_id','smd_id','full_name', 'end_date']] #.to_clipboard()

lost_reelection.sort_values(by='smd_id')

,person_id,smd_id,full_name,end_date
279,10019,smd_1B08,Joshua Ryan Mater,2021-01-02 00:00:00-05:00
253,10025,smd_1C03,Ted Guthrie,2021-01-02 00:00:00-05:00
16,10026,smd_1C04,John V. Zottoli,2021-01-02 00:00:00-05:00
218,10030,smd_1C08,Chris Jackson,2021-01-02 00:00:00-05:00
27,10044,smd_2B01,Matthew H. Sampson,2021-01-02 00:00:00-05:00
245,10051,smd_2B09,Ed Hanlon,2021-01-02 00:00:00-05:00
256,10071,smd_2F07,Kevin J. Sylvester,2021-01-02 00:00:00-05:00
43,10072,smd_2F08,Janice Ferebee,2021-01-02 00:00:00-05:00
92,10361,smd_5B05,"John J. Feeley, Jr.",2021-01-02 00:00:00-05:00
263,10180,smd_5E10,Nancy Darlene Jones,2021-01-02 00:00:00-05:00


In [6]:
len(lost_reelection)

22

Ballot winners who need to be added to the Google Sheet. 

In [7]:
results_cand_people.loc[
    results_cand_people.winner & ~results_cand_people.person_id.isin(commissioners.person_id)
    , ['person_id', 'smd_id', 'full_name']
    ].sort_values(by='smd_id') #.to_clipboard(index=False)

,person_id,smd_id,full_name


Write-in winners who need to be added to the Google Sheet

In [8]:
write_in_winners.loc[
    write_in_winners.person_id.notnull() &
    ~write_in_winners.person_id.isin(commissioners.person_id)
    , ['person_id', 'smd_id', 'write_in_winner_name']
].sort_values(by='smd_id') #.to_clipboard(index=False)

,person_id,smd_id,write_in_winner_name


Districts that had no winner in 2020

In [14]:
write_in_winners['SMD'] = write_in_winners.smd_id.str.replace('smd_','')

In [17]:
', '.join(sorted(write_in_winners[write_in_winners.person_id.isnull()]['SMD'].tolist()))

'2A08, 5A04, 7E03, 7F07, 8B07, 8E03'